In [1]:
%load_ext autoreload
%autoreload 2

# Correlate mRNAs to RBS data

# Load RBS calculator stuff

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

import re
from common import count_ARN_motifs, count_AAN_motifs, count_ARNn


top_dir = os.path.join('data', '22_analyse_RBS_data_Reis')
# Using Reis data
fn = os.path.join('data', 'RBS_Calculator', 'sb0c00394_si_002.xlsx')
excel_file = pd.ExcelFile(fn)
name_sheets = excel_file.sheet_names

df = excel_file.parse(name_sheets[5], index_col=0)
df_aux = excel_file.parse(name_sheets[4], index_col=0)
df['used_mRNA_sequence'] = df_aux['used_mRNA_sequence']
df['predicted_5pUTR_2.0'] = df_aux['predicted_5pUTR']
df['predicted_CDS_2.0'] = df_aux['predicted_CDS']
df['TIR'] = df_aux['TIR']

df = df[df['used_mRNA_sequence'].apply(lambda x: type(x) == str)]
df = df[df['predicted_5pUTR_2.0'].apply(lambda x: type(x) == str)]

# df['ARN count'] = df['used_mRNA_sequence'].apply(count_ARN_motifs)
# df['AAN count'] = df['used_mRNA_sequence'].apply(count_AAN_motifs)
df["ARN count (5' UTR)"] = df['predicted_UTR'].apply(count_ARN_motifs)
df["AAN count (5' UTR)"] = df['predicted_UTR'].apply(count_AAN_motifs)
df["ARNn count (5' UTR)"] = df['predicted_UTR'].apply(count_ARNn)


print("\nShape of dataframe:", df.shape)


Shape of dataframe: (984, 43)


# Load mRNA sequences 

In [3]:
fn_gfp = 'data/RBS_Calculator/RBS_Calc_09_10_2025_08_49_05-GFPmut3.csv'
df_gfp = pd.read_csv(fn_gfp)
fn_cyRFP = 'data/RBS_Calculator/RBS_Calc_09_10_2025_09_22_05-cyRFP1_NewSequence.csv'
df_cyRFP = pd.read_csv(fn_cyRFP)

df_gfp

,start_position,ORF_number,tir,dG_total,dG_mRNA_rRNA,dG_spacing,dG_stacking,dG_standby,dG_start,dG_mRNA,warnings
0,0,0,0.841500,16.198600,-1.270000,0.000000,0.0000,1.735860e+01,-2.76,-2.870000,NaN
1,30,0,2.106833,14.159348,1.158649,0.288000,0.2791,1.533600e+00,-0.42,-11.320000,NaN
2,33,0,8.332874,11.104048,-1.081351,1.525000,-0.2564,1.680000e-02,-0.42,-11.320000,NaN
3,45,0,9.809041,10.741649,-4.551351,2.400000,-0.3970,0.000000e+00,-0.42,-13.710000,NaN
4,55,1,382.874066,2.599348,-9.711351,0.000000,-0.1693,0.000000e+00,-2.76,-15.240000,NaN
5,61,1,127.704549,5.039075,-8.541351,0.005326,0.0665,7.860000e-02,-2.76,-16.190001,NaN
6,63,0,148.159696,4.708949,-12.491351,1.525000,-0.1733,7.860000e-02,-0.42,-16.190001,NaN
7,84,0,0.042587,22.828248,-11.041351,12.768000,-0.8682,1.089800e+00,-0.42,-21.299999,NaN
8,106,1,112.493664,5.320874,-18.821351,0.005326,0.0665,2.650400e+00,-2.76,-24.180000,NaN
9,115,1,71.497911,6.327949,-19.221351,0.672000,0.2273,1.350000e+00,-2.76,-26.059999,NaN
